In [1]:
import pandas as pd
import numpy as np

In [8]:
#Tast 1: 
#Creating Project Dataframe
project_data = {
    'ID': ['A001','A002','A003','A004','A005','A002','A005','A003','A001','A003','A001','A004','A004','A005'],
    'Project': ['Project 1','Project 2','Project 3','Project 4','Project 5','Project 6','Project 7',
                'Project 8','Project 9','Project 10','Project 11','Project 12','Project 13','Project 14'],
    'Cost': [1002000, 2000000, 4500000, 5500000, None, 680000, 400000, 350000, None, 300000, 2000000, 1000000, 3000000, 200000],
    'Status': ['Finished', 'Ongoing', 'Finished', 'Ongoing', 'Finished', 'Failed', 'Finished', 'Failed', 'Ongoing', 'Finished', 'Failed', 'Ongoing', 'Finished', 'Finished']
}
df_project = pd.DataFrame(project_data)
df_project.to_csv('project.csv', index=False)

In [9]:
#Creating Employee Dataframe
employee_data = {
    'ID': ['A001', 'A002', 'A003', 'A004', 'A005'],
    'Name': ['John Alter', 'Alice Luxumberg', 'Tom Sabestine', 'Nina Adgra', 'Amy Johny'],
    'Gender': ['M', 'F', 'M', 'F', 'F'],
    'City': ['Paris', 'London', 'Berlin', 'Newyork', 'Madrid'],
    'Age': [25, 27, 29, 31, 30]
}
df_employee = pd.DataFrame(employee_data)
df_employee.to_csv('employee.csv', index=False)

In [10]:
#Creating Seniority Level DataFrame
seniority_data = {
    'ID': ['A001', 'A002', 'A003', 'A004', 'A005'],
    'Designation Level': [2, 2, 3, 2, 3]
}
df_seniority = pd.DataFrame(seniority_data)
df_seniority.to_csv('seniority.csv', index=False)

In [16]:
#Task 2:
#Replacing missing values with running average using for loop
# Load project.csv
df_project = pd.read_csv('project.csv')

total = 0
count = 0

for i in range(len(df_project)):
    if pd.isnull(df_project.loc[i, 'Cost']):
        if count == 0:
            df_project.loc[i, 'Cost'] = 0
        else:
            df_project.loc[i, 'Cost'] = total / count
    total += df_project.loc[i, 'Cost']
    count += 1

df_project.to_csv('project.csv', index=False)

In [19]:
#Task 3:
#Splitting name columns as 'FistName' and 'LastName" and removing the name column
# Load employee.csv
df_employee = pd.read_csv('employee.csv')

# Safely split name column into two parts
split_names = df_employee['Name'].str.split(' ', n=1, expand=True)

# Assign to new columns
df_employee['First Name'] = split_names[0]

# Handle any missing last names
df_employee['Last Name'] = split_names[1].fillna('')  

# Drop old 'Name' column
df_employee.drop(columns=['Name'], inplace=True)

df_employee.to_csv('employee.csv', index=False)

In [20]:
#Task 4:
#Joining all Dataframes
#Load seniority.csv
df_seniority = pd.read_csv('seniority.csv')

# Merging
final_df = df_project.merge(df_employee, on='ID', how='left')
final_df = final_df.merge(df_seniority, on='ID', how='left')
final_df.to_csv('final.csv', index=False)

In [21]:
#Task 5:
#Adding 5% bonus in final Dataframe if finished
#Load final.csv
final_df = pd.read_csv('final.csv')

final_df['Bonus'] = final_df.apply(lambda row: 0.05 * row['Cost'] if row['Status'] == 'Finished' else 0, axis=1)

In [22]:
#Task 6:
#Demoteing the designation for failed projects
final_df.loc[final_df['Status'] == 'Failed', 'Designation Level'] += 1

# Remove records with designation > 4
final_df = final_df[final_df['Designation Level'] <= 4]

In [27]:
#Task 7:
#Adding Mr. and Mrs. and drop the Gender
def add_title(row):
    return 'Mr. ' + row['First Name'] if row['Gender'] == 'M' else 'Mrs. ' + row['First Name']

final_df['First Name'] = final_df.apply(add_title, axis=1)
final_df.drop(columns=['Gender'], inplace=True)

In [29]:
#Task 8:
#Promote Designation for Age > 29
final_df.loc[final_df['Age'] > 29, 'Designation Level'] = final_df['Designation Level'] - 1
final_df['Designation Level'] = final_df['Designation Level'].clip(lower=1)

In [30]:
#Task 9:
#Finding Total Project cost per employee
total_proj_cost = final_df.groupby(['ID', 'First Name'])['Cost'].sum().reset_index()
total_proj_cost.rename(columns={'Cost': 'Total Cost'}, inplace=True)
total_proj_cost.to_csv('total_project_cost.csv', index=False)

In [31]:
#Task 10:
#Filter Employees with City Containing 'o'
employees_with_o = final_df[final_df['City'].str.contains('o', case=False)]
print(employees_with_o)

      ID     Project       Cost    Status     City  Age  First Name  \
1   A002   Project 2  2000000.0   Ongoing   London   27  Mrs. Alice   
3   A004   Project 4  5500000.0   Ongoing  Newyork   31   Mrs. Nina   
5   A002   Project 6   680000.0    Failed   London   27  Mrs. Alice   
11  A004  Project 12  1000000.0   Ongoing  Newyork   31   Mrs. Nina   
12  A004  Project 13  3000000.0  Finished  Newyork   31   Mrs. Nina   

    Last Name  Designation Level     Bonus  
1   Luxumberg                  2       0.0  
3       Adgra                  1       0.0  
5   Luxumberg                  3       0.0  
11      Adgra                  1       0.0  
12      Adgra                  1  150000.0  
